## Exporting a ResNet50 model to TensorFlow
You can use the Fireball's [exportToTf](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToTf) function to export a model to TensorFlow code. This function creates a python file that implements the model using TensorFlow APIs. It also creates a numpy file (npz) that contains the parameters of the network. This notebook shows how to use this function to export a Fireball model to TensorFlow. It is assumed that a trained ResNet50 model already exists in the ```Models``` directory. Please refer to the notebook [Image Classification with ResNet50](ResNet50.ipynb) for more info about using a pretrained ResNet50 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)
- [Pruning ResNet50 Model](ResNet50-Prune.ipynb)
- [Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)


## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus="upto4"

testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)

orgFileName = "Models/ResNet50RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained


model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.initSession()
model.printLayersInfo()
results = model.evaluate()


Reading from "Models/ResNet50RRPRQR.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      7,114      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      48,512     
S2_L2_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      48,277     
S2_L3_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      48,808     
S3_L1_RES2c1     56 56 256     2 P

## Export the model
Fireball creates a folder and puts 2 files in the folder. Here we call the [exportToTf](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToTf) funtion to export the model.

In [2]:
model.exportToTf("Models/ResNet50_TF", runQuantized=True, classNames=ImageNetDSet.classNames)


Exporting to TensorFlow model "Models/ResNet50_TF" ... 
    Processed all 21 layers.                                      
    Creating parameters file "Params.npz" ... Done.
Done.


## Running inference on the exported model
To verify the exported model, we can now run inference on it. Here we load an image and do the required pre-processing before passing it to the exported model as input. We then print the top-3 most probable predicted labels for the image.

**NOTE**: Please reset the kernel before running the next cell.

In [3]:
import cv2
import numpy as np
imageFileName = 'CoffeeMug.jpg'

img = cv2.imread(imageFileName)     # Reads image in BGR order

# Resize the image to 256x256
imgSize = img.shape[:2]
ratio = 256.0/min(imgSize)
newSize = (int(np.round(imgSize[1]*ratio)), int(np.round(imgSize[0]*ratio)))

# Note: INTER_AREA is best when shrinking and CV_INTER_CUBIC is best when enlarging
img = cv2.resize(img, newSize,  interpolation = (cv2.INTER_AREA if ratio<1.0 else cv2.INTER_CUBIC))

# Now crop the center 224x224 image
dw = newSize[0] - 224
dh = newSize[1] - 224
resizedImg = img[dh//2:dh//2+224, dw//2:dw//2+224,:]

# Normalize the image using the mean values for blue, green, and red
inputImage = (np.float32(resizedImg) - [103.939, 116.779, 123.68])

from Models.ResNet50_TF.TfModel import Network

net=Network()
classNames = net.getClassNames()

classProbs = net.infer([inputImage])[0]
top3Indexes = np.argsort(classProbs)[-3:][::-1]
top3Porbs = classProbs[top3Indexes]
print('Top-3 Classes (For "%s"):'%(imageFileName))
for i in range(3):
    print('    %s (%f)'%(classNames[top3Indexes[i]], top3Porbs[i])) 

Top-3 Classes (For "CoffeeMug.jpg"):
    coffee_mug (0.569166)
    cup (0.259136)
    pitcher (0.098547)


## Also look at

[Exporting ResNet50 Model to CoreML](ResNet50-CoreML.ipynb)

[Exporting ResNet50 Model to ONNX](ResNet50-ONNX.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with ResNet50](ResNet50.ipynb)

[Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)

[Pruning ResNet50 Model](ResNet50-Prune.ipynb)

[Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)
